In [1]:
import cv2
import os
import math
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf

In [2]:
##### Create Train and Test Dataset CSVs or DataFrames
ROOT_DATA_DIR = "../raw_data/Mov_Files"
OUT_DATA_DIR = "../raw_data/Out_Frames"
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]
EXTs_LIST = ["mov"]

def get_data_dict(data_path, class_list, ext_list):
    data_dict = {"vid_paths":[], "vid_files":[], "labels":[]}
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_vids = [l for l in os.listdir(dir_path) if l.split(".")[-1] in ext_list]
        print(f"Found {len(list_vids)} Videos in {dir_path} Directory")
        data_dict["vid_files"].extend(list_vids)
        data_dict["labels"].extend([i for k in range(len(list_vids))])
        data_dict["vid_paths"].extend([dir_path for _ in range(len(list_vids))])
    
    total_videos = len(data_dict["vid_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of Videos : {total_videos}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data_dict = pd.DataFrame(get_data_dict(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST,
                               ext_list=EXTs_LIST))


test_data_dict = pd.DataFrame(get_data_dict(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST,
                               ext_list=EXTs_LIST))       

Found 40 Videos in ../raw_data/Mov_Files/Train/Bad Directory
Found 40 Videos in ../raw_data/Mov_Files/Train/Good Directory
Number of Videos : 80
Number of Labels : 80
Found 10 Videos in ../raw_data/Mov_Files/Test/Bad Directory
Found 10 Videos in ../raw_data/Mov_Files/Test/Good Directory
Number of Videos : 20
Number of Labels : 20


In [3]:
train_data_dict.sample(frac=1).head()

,vid_paths,vid_files,labels
2,../raw_data/Mov_Files/Train/Bad,70.mov,0
16,../raw_data/Mov_Files/Train/Bad,69.mov,0
65,../raw_data/Mov_Files/Train/Good,29.mov,1
14,../raw_data/Mov_Files/Train/Bad,45.mov,0
11,../raw_data/Mov_Files/Train/Bad,75.mov,0


In [4]:
test_data_dict.sample(frac=1).head()

,vid_paths,vid_files,labels
14,../raw_data/Mov_Files/Test/Good,86.mov,1
11,../raw_data/Mov_Files/Test/Good,90.mov,1
6,../raw_data/Mov_Files/Test/Bad,91.mov,0
13,../raw_data/Mov_Files/Test/Good,89.mov,1
16,../raw_data/Mov_Files/Test/Good,84.mov,1


In [5]:
#### Function to convert Videos into Frames
def videos_to_frame(dataframe, out_data_path, class_list, max_frames):
    for index, row in dataframe.iterrows():
        video_to_convert = os.path.join(row["vid_paths"], row["vid_files"])
        out_frames_path = os.path.join(out_data_path, class_list[row["labels"]], row["vid_files"].split(".")[0])
        try:
            os.makedirs(out_frames_path, exist_ok=True)
        except:
            print("File Already Created")
        print(f"Converting {video_to_convert} and Storing Frames in {out_frames_path}")
        cap=cv2.VideoCapture(video_to_convert)
        
        while(cap.isOpened()):
            frameId = cap.get(1)
            ret, frame = cap.read()
            if (ret != True):
                break
            filename =f"frame-{str(int(frameId)).zfill(7)}.jpg"
            cv2.imwrite(os.path.join(out_frames_path, filename), frame)
        cap.release()
        
        

videos_to_frame(dataframe=train_data_dict,
                out_data_path=os.path.join(OUT_DATA_DIR, DATA_SUB_DIRs[0]),
                class_list=CLS_LIST,
                max_frames=100)

videos_to_frame(dataframe=test_data_dict,
                out_data_path=os.path.join(OUT_DATA_DIR, DATA_SUB_DIRs[1]),
                class_list=CLS_LIST,
                max_frames=100)

Converting ../raw_data/Mov_Files/Train/Bad/74.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/74
Converting ../raw_data/Mov_Files/Train/Bad/53.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/53
Converting ../raw_data/Mov_Files/Train/Bad/70.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/70
Converting ../raw_data/Mov_Files/Train/Bad/62.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/62
Converting ../raw_data/Mov_Files/Train/Bad/65.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/65
Converting ../raw_data/Mov_Files/Train/Bad/44.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/44
Converting ../raw_data/Mov_Files/Train/Bad/56.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/56
Converting ../raw_data/Mov_Files/Train/Bad/71.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/71
Converting ../raw_data/Mov_Files/Train/Bad/80.mov and Storing Frames in ../raw_data/Out_Frames/Train/Bad/80
Converting ../raw_data/Mov_F

In [ ]:
video_capturing_function(train_set,"Train_Videos")
video_capturing_function(test_set,"Test_Videos")

In [ ]:
def data_load_function_10frames(dataset,directory):
    frames=[]
    for i in np.arange(len(dataset)):
        vid_name=dataset.video_name[i].split(".")[0]
        vid_dir_path=os.path.join(directory,vid_name)
        frames_to_select=[]
        for l in np.arange(0,10):
            frames_to_select.append('frame%d.jpg' % l)
        vid_data=[]
        for frame in frames_to_select:
            image=Image.open(os.path.join(vid_dir_path,frame))
            image=image.resize((250, 250), Image.ANTIALIAS)
            datu=np.asarray(image)
            normu_dat=datu/255
            vid_data.append(normu_dat)
        vid_data=np.array(vid_data)
        frames.append(vid_data)
    return np.array(frames)

In [ ]:
train_vid_dat = pd.concat([train_set.iloc[:30,:],train_set.iloc[50:,:]])

In [ ]:
validation_vid_dat = train_set.iloc[30:50,:]

In [ ]:
validation_vid_dat = validation_vid_dat.reset_index()

In [ ]:
train_vid_dat = train_vid_dat.reset_index()

In [ ]:
validation_vid_dat = validation_vid_dat[['video_name','label']]

In [ ]:
train_vid_dat = train_vid_dat[['video_name','label']]

In [ ]:
y = train_vid_dat['label']

In [ ]:
test_dataset_new=data_load_function_10frames(test_set,'output/Test_Videos/')
train_dataset_new=data_load_function_10frames(train_vid_dat,'output/Train_Videos/')
validation_dataset_new=data_load_function_10frames(validation_vid_dat,'output/Val_Videos/')

In [ ]:
# Reshaping tensors to confirm with the model we are going to train:
test_dataset_new=test_dataset_new.reshape((20,10,250,250,1))
train_dataset_new=train_dataset_new.reshape((60,10,250,250,1))
validation_dataset_new=validation_dataset_new.reshape((20,10,250,250,1))

In [ ]:
!pip install keras

In [ ]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
model_cnlst = models.Sequential()
model_cnlst.add(TimeDistributed(Conv2D(128, (3, 3), strides=(1,1),activation='relu'),input_shape=(10, 250, 250, 1)))
model_cnlst.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(MaxPooling2D(2,2)))
model_cnlst.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(Conv2D(32, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(MaxPooling2D(2,2)))
model_cnlst.add(TimeDistributed(BatchNormalization()))


model_cnlst.add(TimeDistributed(Flatten()))
model_cnlst.add(Dropout(0.2))

model_cnlst.add(LSTM(32,return_sequences=False,dropout=0.2)) # used 32 units

model_cnlst.add(Dense(64,activation='relu'))
model_cnlst.add(Dense(32,activation='relu'))
model_cnlst.add(Dropout(0.2))
model_cnlst.add(Dense(1, activation='sigmoid'))
model_cnlst.summary()

In [ ]:
import keras

In [ ]:
train_labels = pd.concat([train_set.iloc[:30,:][['label']],train_set.iloc[50:,:][['label']]])

In [ ]:
validation_labels = train_set.iloc[30:50,:]['label']

In [ ]:
from keras import optimizers
callbacks_list_cnlst=[keras.callbacks.EarlyStopping(
monitor='acc',patience=3),
               keras.callbacks.ModelCheckpoint(
               filepath='cnn_lstm_model_new3.h5',
               monitor='val_loss',
               save_best_only=True),
                keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3)
               ]


optimizer_new=optimizers.RMSprop(lr=0.01)
model_cnlst.compile(optimizer=optimizer_new,loss='binary_crossentropy',metrics=['acc'])
# Training:
history_new_cnlst=model_cnlst.fit(train_dataset_new,train_labels,batch_size=10,epochs=20,
                        validation_data=(validation_dataset_new,validation_labels),
                       callbacks=callbacks_list_cnlst)

In [ ]:
np.mean(history_new_cnlst.history['acc'])